In [1]:
using BenchmarkTools, Compat, DataFrames, Distributions, ForwardDiff, PyCall

In [2]:
# Stochastic gradient descent, mixed logit.

df = readtable("../data/parsed_model_australia.txt", separator = ' ', header = false)

a, b = size(df)

const n_individuals = a
const n_alternatives = 4
const n_parameters = b
const n_simulations = 1000
const simulated_b = 5

srand(123456)

rand_contdist(Dist::Distribution) = quantile(Dist, rand())

rand_contdist (generic function with 1 method)

In [3]:
head(df)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
1,4,1,0,0,0,0,1,0,0,0,0,1,0,35,0,0,0,69,34,35,0,70,71,70,30
2,4,1,0,0,0,0,1,0,0,0,0,1,0,30,0,0,0,64,44,53,0,68,84,85,50
3,4,1,0,0,0,0,1,0,0,0,0,1,0,40,0,0,0,69,34,35,0,129,195,149,101
4,4,1,0,0,0,0,1,0,0,0,0,1,0,70,0,0,0,64,44,53,0,59,79,81,32
5,4,1,0,0,0,0,1,0,0,0,0,1,0,45,0,0,0,64,44,53,0,82,93,94,99
6,2,1,0,0,0,0,1,0,0,0,0,1,0,20,0,0,0,69,40,35,0,70,57,58,43


In [4]:
mixed_logit = zeros(n_individuals, n_simulations)

function simulate()
    for i = 1:n_individuals, j = 1:n_simulations
        mixed_logit[i, j] = rand()
    end
end

simulate()

In [5]:
function individual(θ::Vector, i::Int64)
    m, n = size(df)
    choice = df[i, 1][1]
    alternatives = collect(1:n_alternatives)
    splice!(alternatives, choice)
    
    function utility(β::Vector, k::Int64)
        temp = Float64[]
        k += 1
        while k <= n_parameters
            push!(temp, df[i, k])
            k += n_alternatives
        end
        return dot(temp, β)
    end
    
    function construct(γ::Vector, θ::Vector)
        return θ[1]+θ[2]*γ[1]
    end

    function probability(θ::Vector)
        logit = 0.0
        t = 0.0
        for k = 1:n_simulations
            β = []
            for j = 1:simulated_b-1
                push!(β, θ[j])
            end
            γ = [mixed_logit[i, k]]
            push!(β, construct(γ, θ[simulated_b:simulated_b+1]))
            for j = simulated_b+2:length(θ)
                push!(β, θ[j])
            end 
            c = utility(β, choice)
            for alternative in alternatives
                t += exp(utility(β, alternative)-c)
            end
            logit += 1/(1+t)
        end
        return logit/n_simulations
    end
    
    return probability
end

individual (generic function with 1 method)

In [6]:
function f(β::Vector)
    model = 0.0
    i = 1
    while i <= n_individuals
        probability = individual(β, i)
        model += log(probability(β))
        i += 1
    end
    return -model/n_individuals
end

f (generic function with 1 method)

In [7]:
function g(x::Vector, i::Int64, j::Int64)
    t = zeros(7)
    for k = i:j
        probability = individual(x, k)
        t += (1/probability(x))*ForwardDiff.gradient(probability, x)
    end
    return -t/j
end

function g!(x::Vector, i::Int64, j::Int64, storage::Vector)
    s = g(x, i, j)
    storage[1:length(s)] = s[1:length(s)]
end

g! (generic function with 1 method)

In [8]:
function shuffle!(df::DataFrame)
    for i = size(df, 1):-1:2
        j = rand(1:i)
        df[i, :], df[j, :] = df[j, :], df[i, :]
    end
end

shuffle! (generic function with 1 method)

In [9]:
function sgd(ω0::Vector, η::Float64, batch_size::Int64, tol::Float64 = 1e-6)
    k = 1
    nmax = 1000
    ω = ω0
    n = length(ω)
    dfω = ones(n)
    while dot(dfω, dfω) > tol && k < nmax
        shuffle!(df)
        for i = 1:batch_size:n_individuals
            ω -= η*g!(ω, i, batch_size, dfω)
        end
        k += 1
    end
    return ω, k
end

sgd (generic function with 2 methods)

In [10]:
sgd(zeros(7), 0.001, 1) # pas exact

([-0.000316059, -0.000316059, 0.000948177, -0.00632118, -0.0230723, -0.0120039, 0.00221241], 2)